In [1]:
# #extract/download competition data using kaggle API
# import os, sys, zipfile, glob, pathlib, subprocess
# from kaggle.api.kaggle_api_extended import KaggleApi

# COMP = "nfl-big-data-bowl-2026-analytics"
# DATA_DIR = "data"
# os.makedirs(DATA_DIR, exist_ok=True)

# # (Optional) ensure latest kaggle in THIS env
# # subprocess.run([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "kaggle"], check=True)

# # Auth (kaggle.json in %USERPROFILE%\.kaggle or env vars)
# api = KaggleApi()
# api.authenticate()
# print("✅ Authenticated")

# # --- List files (note the .files!) ---
# resp = api.competition_list_files(COMP)
# if not getattr(resp, "files", None):
#     print("⚠️ No files listed. If this seems wrong, double-check you’ve joined/accepted rules.")
# else:
#     print("📦 Files available:")
#     for f in resp.files:
#         print("  -", f.name, getattr(f, "size", ""))

# # --- Download all (single zip) ---
# print(f"⬇️  Downloading all files for '{COMP}' to '{DATA_DIR}' ...")
# api.competition_download_files(COMP, path=DATA_DIR, quiet=False)

# # --- Unzip into DATA_DIR ---
# for z in glob.glob(os.path.join(DATA_DIR, "*.zip")):
#     print(f"📂 Extracting: {os.path.basename(z)}")
#     with zipfile.ZipFile(z) as zz:
#         zz.extractall(DATA_DIR)
# print("✅ Done")


In [2]:
# #combing the files
# import os, glob, hashlib, pandas as pd
# from pathlib import Path

# # ---- 1) locate the train folder (robust)
# ROOT = Path.cwd()  # or Path(r"X:\path\to\Big_Data_Bowl")
# candidates = [p for p in ROOT.rglob("train") if p.is_dir()]
# if not candidates:
#     raise SystemExit("Couldn't find a 'train' folder under the current project. Set ROOT accordingly.")
# TRAIN_DIR = max(candidates, key=lambda p: len(str(p)))  # pick deepest match
# print(f"[info] Using train dir: {TRAIN_DIR}")

# # ---- 2) discover CSVs
# csvs = sorted(glob.glob(str(TRAIN_DIR / "**/*.csv"), recursive=True))
# if not csvs:
#     raise SystemExit("No CSVs found under train/.")
# print(f"[info] Found {len(csvs)} CSVs")

# # ---- 3) group files by schema (set of column names, case-normalized)
# groups = {}
# for fp in csvs:
#     try:
#         cols = pd.read_csv(fp, nrows=0).columns
#     except Exception as e:
#         print(f"[warn] Skipping unreadable file: {fp} ({e})")
#         continue
#     schema_key = tuple(sorted([c.strip().lower() for c in cols]))
#     groups.setdefault(schema_key, []).append(fp)

# print(f"[info] Discovered {len(groups)} schema group(s).")

# # ---- 4) concatenate each group safely (chunked), write CSV + Parquet
# OUTDIR = TRAIN_DIR.parent / "combined"
# OUTDIR.mkdir(exist_ok=True, parents=True)

# def friendly_name(file_list):
#     # Try to produce a nice name (tracking/plays/players/etc). Fallback to a short hash.
#     names = " ".join([Path(f).name.lower() for f in file_list[:10]])
#     if "tracking" in names:
#         return "tracking_all"
#     if "plays" in names:
#         return "plays_all"
#     if "players" in names:
#         return "players_all"
#     if "tackles" in names:
#         return "tackles_all"
#     if "games" in names:
#         return "games_all"
#     # fallback
#     h = hashlib.md5("|".join(sorted(file_list)).encode()).hexdigest()[:8]
#     return f"group_{h}"

# combined_index = []
# for i, (schema, files) in enumerate(groups.items(), 1):
#     out_base = friendly_name(files)
#     out_csv  = OUTDIR / f"{out_base}.csv"
#     out_parq = OUTDIR / f"{out_base}.parquet"

#     print(f"\n[combine] Group {i}/{len(groups)}: {len(files)} file(s) -> {out_base}")
#     # union all rows with identical column set
#     chunks = []
#     for f in files:
#         # Use low-memory chunks to avoid RAM spikes
#         for chunk in pd.read_csv(f, chunksize=250_000):
#             # standardize column order/casing
#             chunk.columns = [c.strip() for c in chunk.columns]
#             chunk = chunk[[c for c in chunk.columns]]  # keep order
#             chunks.append(chunk)
#     if not chunks:
#         print("[warn] No readable data in this group; skipping.")
#         continue
#     df = pd.concat(chunks, ignore_index=True)

#     # write outputs
#     df.to_csv(out_csv, index=False)
#     try:
#         df.to_parquet(out_parq, index=False)
#     except Exception as e:
#         print(f"[warn] Parquet write failed ({e}); CSV saved.")

#     combined_index.append({"output": out_base, "rows": len(df), "files": len(files)})

# # ---- 5) also bring along supplementary_data.csv if present (leave separate)
# supp = list((TRAIN_DIR.parent).glob("supplementary_data.csv"))
# if supp:
#     dest = OUTDIR / "supplementary_data.csv"
#     pd.read_csv(supp[0]).to_csv(dest, index=False)
#     print(f"\n[info] Copied supplementary_data.csv -> {dest}")

# # ---- 6) show a small index of what we built
# idx = pd.DataFrame(combined_index).sort_values("rows", ascending=False)
# print("\n[summary] Combined files:")
# display(idx)
# print(f"\nOutputs in: {OUTDIR}")


In [3]:
from pathlib import Path
import pandas as pd, numpy as np, re

COMBINED_DIR = next(p for p in Path.cwd().rglob("combined") if p.is_dir())
print("Using combined dir:", COMBINED_DIR)

# find group files
group_files = sorted(list(COMBINED_DIR.glob("group_*.csv")))
assert group_files, "No group_*.csv found in combined/"

dfs = {f.stem: pd.read_csv(f, low_memory=False) for f in group_files}
for k,v in dfs.items():
    print(k, v.shape, list(v.columns)[:8], "...")

# heuristics: rich has s,a,dir,o and ball_land_*; light is just game_id,play_id,nfl_id,frame_id,x,y
def is_rich(df):
    need = {"s","a","dir","o","ball_land_x","ball_land_y"}
    return need.issubset(set(df.columns))

rich_name, light_name = None, None
for name, df in dfs.items():
    if is_rich(df):
        rich_name = name
    else:
        # minimal schema check
        cols = {"game_id","play_id","nfl_id","frame_id","x","y"}
        if cols.issubset(set(df.columns)):
            light_name = name

assert rich_name, "Couldn't find a 'rich' group (with s,a,dir,o,ball_land_x/ball_land_y)."
print(f"[info] rich={rich_name}, light={light_name}")
rich = dfs[rich_name].copy()
light = dfs[light_name].copy() if light_name else None


Using combined dir: c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\TESM-501\Big_Data_Bowl\Big_Data_Bowl\data\114239_nfl_competition_files_published_analytics_final\combined
group_75913e02 (4880579, 23) ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'frame_id', 'play_direction', 'absolute_yardline_number', 'player_name'] ...
group_a3a1cd60 (562936, 6) ['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y'] ...
[info] rich=group_75913e02, light=group_a3a1cd60


In [4]:
import numpy as np

def to_num(s):
    return pd.to_numeric(s, errors="coerce")

# basic types
for c in ["game_id","play_id","nfl_id","frame_id"]:
    if c in rich.columns:
        rich[c] = to_num(rich[c])

# keep only rows with key fields
keep_cols = ["game_id","play_id","nfl_id","frame_id","x","y","s","a","dir","o",
             "player_to_predict","player_name","player_height","player_weight",
             "player_birth_date","player_position","player_side","player_role",
             "play_direction","absolute_yardline_number","num_frames_output",
             "ball_land_x","ball_land_y"]
keep_cols = [c for c in keep_cols if c in rich.columns]
df = rich[keep_cols].dropna(subset=["game_id","play_id","nfl_id","frame_id","x","y","ball_land_x","ball_land_y"]).copy()

# geometry toward landing point
dx = df["ball_land_x"] - df["x"]
dy = df["ball_land_y"] - df["y"]
df["dist_to_land"] = np.hypot(dx, dy)
df["bearing_to_land_deg"] = np.degrees(np.arctan2(dy, dx)) % 360

# helper for smallest angle difference in degrees
def ang_err(a, b):
    d = (a - b + 180) % 360 - 180
    return d

# dir = movement direction in degrees (0-360)
df["heading_error_deg"] = ang_err(df["bearing_to_land_deg"], df["dir"])
df["heading_align_cos"] = np.cos(np.radians(df["heading_error_deg"]))  # 1 = perfectly toward landing

# per-player/play ordering
df = df.sort_values(["game_id","play_id","nfl_id","frame_id"])

# convergence rate (how quickly distance-to-land decreases per frame)
df["dist_next"] = df.groupby(["game_id","play_id","nfl_id"])["dist_to_land"].shift(-1)
df["converge_rate"] = (df["dist_to_land"] - df["dist_next"])  # >0 means moving closer between frames
# we don't know frame rate; treat per-frame unit. It's fine; we're standardizing later.

# basic scalers / bins for context
df["ytg_bin"] = pd.cut(df["absolute_yardline_number"].fillna(df["absolute_yardline_number"].median()),
                       bins=[-1,20,40,60,80,120], labels=["own20","20-40","40-60","60-80","redz"])
df["frame_bin"] = pd.cut(df["frame_id"], bins=[-1,10,20,40,80,9999], labels=["t0-10","11-20","21-40","41-80","80+"])

# small cleanups
for c in ["s","a"]:
    if c in df.columns:
        df[c] = to_num(df[c])
for c in ["player_position","player_role","player_side","play_direction"]:
    if c in df.columns:
        df[c] = df[c].astype("category")


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor

target = "converge_rate"
# rows with valid target
mod = df[df[target].notna()].copy()

num_feats = [c for c in ["s","a","dist_to_land","frame_id","absolute_yardline_number"] if c in mod.columns]
cat_feats = [c for c in ["player_position","player_role","player_side","play_direction","ytg_bin","frame_bin"] if c in mod.columns]

X = mod[num_feats + cat_feats]
y = mod[target].astype(float)

pre = ColumnTransformer(
    transformers=[
        ("num","passthrough", num_feats),
        ("cat", OneHotEncoder(handle_unknown="ignore", min_frequency=0.01), cat_feats)
    ],
    remainder="drop"
)
reg = Pipeline([("pre", pre), ("gbr", GradientBoostingRegressor(random_state=42))])
reg.fit(X, y)

# predict for all frames we can
df["_can_pred"] = df[num_feats + cat_feats].notna().all(axis=1)
df["E_converge"] = np.nan
df.loc[df["_can_pred"], "E_converge"] = reg.predict(df.loc[df["_can_pred"], num_feats + cat_feats])

# residual = actual - expected
df["IQ_frame_raw"] = df[target] - df["E_converge"]

# standardize by context to get a z-score-ish IQ
ctx_cols = ["frame_bin","ytg_bin","player_position"]
ctx = df.copy()
g = ctx.groupby(ctx_cols, dropna=False)["IQ_frame_raw"]
std_ctx = g.transform(lambda s: s.std(ddof=0) if s.notna().sum() > 40 else np.nan)

global_std = ctx["IQ_frame_raw"].std(ddof=0)
std_ctx = std_ctx.fillna(global_std if not np.isnan(global_std) else 1.0)
df["IQ_frame"] = df["IQ_frame_raw"] / std_ctx.replace(0, np.nan)
df["IQ_frame"] = df["IQ_frame"].fillna(0.0)

df[["game_id","play_id","nfl_id","frame_id","converge_rate","E_converge","IQ_frame","heading_align_cos"]].head()


C:\Users\rajas\AppData\Local\Temp\ipykernel_19944\3467958568.py:37: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = ctx.groupby(ctx_cols, dropna=False)["IQ_frame_raw"]


,game_id,play_id,nfl_id,frame_id,converge_rate,E_converge,IQ_frame,heading_align_cos
182,2023090700,101,43290,1,0.000000,-0.058663,0.894288,-0.420835
183,2023090700,101,43290,2,-0.006500,-0.058663,0.795205,-0.396785
184,2023090700,101,43290,3,-0.006501,-0.058663,0.795183,-0.229819
185,2023090700,101,43290,4,-0.006502,-0.058663,0.795161,0.999992
186,2023090700,101,43290,5,-0.013009,-0.063001,0.762101,0.779825


In [6]:
df.to_csv(Path("Regression_model_output.csv"))

In [11]:
import pandas as pd
import numpy as np

# --- keep only needed cols & downcast (same as before) ---
cols = ["game_id","play_id","nfl_id","player_name","player_position",
        "frame_id","IQ_frame","heading_align_cos","dist_to_land"]
df_small = df.loc[:, [c for c in cols if c in df.columns]].copy()

for c in ["frame_id"]:
    if c in df_small: df_small[c] = pd.to_numeric(df_small[c], downcast="integer")
for c in ["IQ_frame","heading_align_cos","dist_to_land"]:
    if c in df_small: df_small[c] = pd.to_numeric(df_small[c], downcast="float")
for c in ["player_position","player_name"]:
    if c in df_small: df_small[c] = df_small[c].astype("category")

keys = [k for k in ["game_id","play_id","nfl_id","player_name","player_position"] if k in df_small.columns]

gb = df_small.groupby(keys, sort=False, observed=True)

# --- single-pass named aggregation (avoids concat/MultiIndex union) ---
agg_core = (gb.agg(
    IQ_mean=('IQ_frame','mean'),
    align_mean=('heading_align_cos','mean'),
    frames=('frame_id','count'),
    dist_start=('dist_to_land','first'),
    dist_end=('dist_to_land','last'),
).reset_index())

# --- p90 separately, then merge on keys (no MultiIndex issues) ---
try:
    p90 = (gb['IQ_frame']
           .quantile(0.9, interpolation='linear')
           .reset_index(name='IQ_p90'))
    iq_play_player = agg_core.merge(p90, on=keys, how='left')
except Exception:
    iq_play_player = agg_core
    iq_play_player['IQ_p90'] = np.nan

# --- volume metric ---
iq_play_player['IQ_volume'] = iq_play_player['IQ_mean'] * np.log1p(iq_play_player['frames'])

# --- play-level team intent: top-5 by IQ_mean without global sort ---
iq_play_player['rank_in_play'] = (
    iq_play_player.groupby(['game_id','play_id'], observed=True)['IQ_mean']
    .rank(method='first', ascending=False)
)
topN = iq_play_player.loc[iq_play_player['rank_in_play'] <= 5, ['game_id','play_id','IQ_mean','frames']]

iq_play = (topN.groupby(['game_id','play_id'], observed=True, sort=False)
           .agg(IQ_team_play=('IQ_mean','mean'),
                frames_total=('frames','sum'))
           .reset_index())

# --- save ---
OUT = COMBINED_DIR / "iq_outputs"
OUT.mkdir(parents=True, exist_ok=True)
iq_play_player.to_csv(OUT/"iq_play_player.csv")
iq_play.to_csv(OUT/"iq_play.csv")
iq_play_player.to_parquet(OUT / "iq_play_player.parquet", index=False)
iq_play.to_parquet(OUT / "iq_play.parquet", index=False)
print("[done] wrote:", OUT)


[done] wrote: c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\TESM-501\Big_Data_Bowl\Big_Data_Bowl\data\114239_nfl_competition_files_published_analytics_final\combined\iq_outputs


In [12]:
# === Enrich frame table `df` with columns from supplementary_data.csv ===
import pandas as pd
import numpy as np
from pathlib import Path

# 1) locate supplementary_data.csv (your screenshot shows two copies)
sup_paths = [p for p in Path.cwd().rglob("supplementary_data.csv")]
if not sup_paths:
    raise SystemExit("Could not find supplementary_data.csv")
SUP = sup_paths[0]
print("[info] using:", SUP)

sup = pd.read_csv(SUP, low_memory=False)

# 2) pick columns for mapping (safe if some are missing)
keep = [c for c in [
    "game_id","play_id","season","week","game_date","quarter","game_clock",
    "possession_team","defensive_team","home_team_abbr","visitor_team_abbr",
    "down","yards_to_go","yardline_side","yardline_number",
    "play_action","dropback_type","dropback_distance","pass_location_type",
    "defenders_in_the_box","team_coverage_man_zone","team_coverage_type",
    "yards_gained","expected_points","expected_points_added"
] if c in sup.columns]

sup_map = (sup[keep]
    .drop_duplicates(subset=["game_id","play_id"])
    .copy())

# normalize id types for robust joins
for c in ["game_id","play_id"]:
    if c in sup_map.columns:
        sup_map[c] = pd.to_numeric(sup_map[c], errors="coerce").astype("Int64").astype(str)

for c in ["game_id","play_id"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64").astype(str)

# 3) merge onto frames
df_enriched = df.merge(sup_map, on=["game_id","play_id"], how="left")

# 4) write enriched frames for downstream use
OUT = next(p for p in Path.cwd().rglob("combined")) / "iq_outputs"
OUT.mkdir(parents=True, exist_ok=True)
df_enriched.to_parquet(OUT / "iq_frames_enriched.parquet", index=False)
print("[done] wrote:", OUT / "iq_frames_enriched.parquet")

# 5) (optional) rebuild play-level intent with team/season/week attached
#    If you already computed iq_play_player / iq_play, just re-merge meta onto them.
def load_if_exists(p): 
    return pd.read_parquet(p) if Path(p).exists() else None

p_play_player = OUT / "iq_play_player.parquet"
p_play        = OUT / "iq_play.parquet"

iq_play_player = load_if_exists(p_play_player)
iq_play        = load_if_exists(p_play)

if iq_play_player is not None:
    for c in ["game_id","play_id"]:
        if c in iq_play_player.columns:
            iq_play_player[c] = pd.to_numeric(iq_play_player[c], errors="coerce").astype("Int64").astype(str)
    iq_play_player = iq_play_player.merge(sup_map, on=["game_id","play_id"], how="left")
    iq_play_player.to_parquet(OUT / "iq_play_player_enriched.parquet", index=False)
    iq_play_player.to_csv(OUT / "iq_play_player_enriched.csv", index=False)
    print("[done] wrote:", OUT / "iq_play_player_enriched.parquet")

if iq_play is not None:
    for c in ["game_id","play_id"]:
        if c in iq_play.columns:
            iq_play[c] = pd.to_numeric(iq_play[c], errors="coerce").astype("Int64").astype(str)
    iq_play = iq_play.merge(sup_map, on=["game_id","play_id"], how="left")
    iq_play.to_parquet(OUT / "iq_play_enriched.parquet", index=False)
    iq_play.to_csv(OUT / "iq_play_enriched.csv", index=False)
    print("[done] wrote:", OUT / "iq_play_enriched.parquet")

# 6) quick sanity: IQ vs outcomes if present  (robust to column-name variants)

def _first_present(df, candidates):
    """Return the first column name that exists in df from a list of candidates, else None."""
    for c in candidates:
        if c in df.columns:
            return c
    return None

# common aliases across BDB vintages
YARDS_CANDS = [
    "yards_gained", "pre_penalty_yards_gained", "pre_penalty_yards",
    "gain", "return_yards", "punt_return_yards", "kick_return_yards"
]
EPA_CANDS = ["expected_points_added", "epa", "EPA", "ep_added"]

yards_col = _first_present(sup_map, YARDS_CANDS)
epa_col   = _first_present(sup_map, EPA_CANDS)

print("[info] yards-like column ->", yards_col)
print("[info] EPA-like column   ->", epa_col)

if iq_play is None:
    print("[note] iq_play not found; run the IQ aggregation step first.")
else:
    merge_cols = ["game_id", "play_id"]
    if yards_col: merge_cols.append(yards_col)
    if epa_col:   merge_cols.append(epa_col)

    tmp = iq_play.merge(sup_map[merge_cols], on=["game_id","play_id"], how="left")

    # standardize names for easy correlation code
    rename_map = {}
    if yards_col: rename_map[yards_col] = "yards_std"
    if epa_col:   rename_map[epa_col]   = "epa_std"
    tmp = tmp.rename(columns=rename_map)

    # correlations (only compute if columns exist)
    if "IQ_team_play" not in tmp.columns:
        print("[warn] 'IQ_team_play' is missing in iq_play; check your aggregation step.")
    else:
        if "yards_std" in tmp.columns:
            num = tmp[["IQ_team_play", "yards_std"]].dropna()
            if len(num):
                r = num.corr(method="pearson").iloc[0,1]
                print("[corr] IQ_team_play vs yards:", round(float(r), 3), " (n=", len(num), ")")
            else:
                print("[note] Not enough overlap to correlate IQ vs yards.")
        else:
            print("[note] No yards-like column found among:", YARDS_CANDS)

        if "epa_std" in tmp.columns:
            num = tmp[["IQ_team_play", "epa_std"]].dropna()
            if len(num):
                r = num.corr(method="pearson").iloc[0,1]
                print("[corr] IQ_team_play vs EPA:", round(float(r), 3), " (n=", len(num), ")")
            else:
                print("[note] Not enough overlap to correlate IQ vs EPA.")
        else:
            print("[note] No EPA-like column found among:", EPA_CANDS)

[info] using: c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\TESM-501\Big_Data_Bowl\Big_Data_Bowl\data\114239_nfl_competition_files_published_analytics_final\supplementary_data.csv
[done] wrote: c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\TESM-501\Big_Data_Bowl\Big_Data_Bowl\data\114239_nfl_competition_files_published_analytics_final\combined\iq_outputs\iq_frames_enriched.parquet
[done] wrote: c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\TESM-501\Big_Data_Bowl\Big_Data_Bowl\data\114239_nfl_competition_files_published_analytics_final\combined\iq_outputs\iq_play_player_enriched.parquet
[done] wrote: c:\Users\rajas\Documents\ADS\Desktop\Applied Data Science\TESM-501\Big_Data_Bowl\Big_Data_Bowl\data\114239_nfl_competition_files_published_analytics_final\combined\iq_outputs\iq_play_enriched.parquet
[info] yards-like column -> yards_gained
[info] EPA-like column   -> expected_points_added
[note] No yards-like column found among: ['yards_gained', 'pre_pen

In [18]:
iq_play_player.head(10)

,game_id,play_id,nfl_id,player_name,player_position,IQ_mean,align_mean,frames,dist_start,dist_end,...,play_action,dropback_type,dropback_distance,pass_location_type,defenders_in_the_box,team_coverage_man_zone,team_coverage_type,yards_gained,expected_points,expected_points_added
0,2023090700,101,43290,Jared Goff,QB,0.292423,0.259220,26,39.853405,41.088520,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
1,2023090700,101,44930,Josh Reynolds,WR,-0.394867,-0.403419,26,25.449656,17.986063,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
2,2023090700,101,46137,Justin Reid,SS,1.009647,-0.959142,26,24.078863,19.375389,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
3,2023090700,101,52546,L'Jarius Sneed,CB,0.583084,-0.337763,26,22.288868,19.820143,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
4,2023090700,101,53487,Nick Bolton,MLB,1.348255,-0.892428,26,33.570587,26.305412,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
5,2023090700,101,53541,Amon-Ra St. Brown,WR,0.928232,-0.805932,26,29.783520,21.268497,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
6,2023090700,101,53959,Brock Wright,TE,0.205771,-0.786040,26,41.831234,38.774967,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
7,2023090700,101,54486,Trent McDuffie,CB,-0.272659,-0.563769,26,44.243103,44.342922,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
8,2023090700,101,54527,Bryan Cook,FS,0.571816,-0.857464,26,38.734100,36.314144,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
9,2023090700,194,41325,Jerick McKinnon,RB,-0.120974,0.304713,32,10.373141,4.068412,...,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN,10,0.121505,1.702563


In [16]:
iq_play.head(10)

,game_id,play_id,IQ_team_play,frames_total,season,week,game_date,quarter,game_clock,possession_team,...,play_action,dropback_type,dropback_distance,pass_location_type,defenders_in_the_box,team_coverage_man_zone,team_coverage_type,yards_gained,expected_points,expected_points_added
0,2023090700,101,0.888207,130,2023,1,09/07/2023,1,14:25,DET,...,False,TRADITIONAL,2.13,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,0,0.927021,-2.145443
1,2023090700,194,0.346120,160,2023,1,09/07/2023,1,12:56,KC,...,False,TRADITIONAL,3.86,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN,10,0.121505,1.702563
2,2023090700,219,0.550743,85,2023,1,09/07/2023,1,12:20,KC,...,False,TRADITIONAL,2.37,INSIDE_BOX,6,ZONE_COVERAGE,COVER_4_ZONE,5,1.824068,0.089352
3,2023090700,361,0.057183,255,2023,1,09/07/2023,1,10:06,DET,...,False,TRADITIONAL,3.03,OUTSIDE_RIGHT,6,ZONE_COVERAGE,COVER_4_ZONE,5,-1.049160,-0.862062
4,2023090700,436,-0.137697,100,2023,1,09/07/2023,1,08:09,DET,...,False,TRADITIONAL,2.55,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN,13,0.312204,1.613927
5,2023090700,461,1.221083,115,2023,1,09/07/2023,1,07:30,DET,...,True,TRADITIONAL,4.72,INSIDE_BOX,7,ZONE_COVERAGE,COVER_6_ZONE,21,1.926131,1.345633
6,2023090700,530,0.194973,100,2023,1,09/07/2023,1,05:36,DET,...,False,TRADITIONAL,2.08,INSIDE_BOX,6,ZONE_COVERAGE,COVER_2_ZONE,5,4.614957,0.031384
7,2023090700,621,0.638741,145,2023,1,09/07/2023,1,02:52,DET,...,False,TRADITIONAL,3.61,INSIDE_BOX,5,MAN_COVERAGE,COVER_1_MAN,9,4.149969,2.850031
8,2023090700,713,0.324336,120,2023,1,09/07/2023,1,02:11,KC,...,False,TRADITIONAL,3.54,INSIDE_BOX,5,ZONE_COVERAGE,COVER_4_ZONE,0,0.948264,-0.873874
9,2023090700,736,0.919642,155,2023,1,09/07/2023,1,02:06,KC,...,False,TRADITIONAL,6.58,INSIDE_BOX,6,MAN_COVERAGE,COVER_1_MAN,19,0.074390,2.717539


In [17]:
iq_play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14108 entries, 0 to 14107
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 14108 non-null  object 
 1   play_id                 14108 non-null  object 
 2   IQ_team_play            14108 non-null  float32
 3   frames_total            14108 non-null  int64  
 4   season                  14108 non-null  int64  
 5   week                    14108 non-null  int64  
 6   game_date               14108 non-null  object 
 7   quarter                 14108 non-null  int64  
 8   game_clock              14108 non-null  object 
 9   possession_team         14108 non-null  object 
 10  defensive_team          14108 non-null  object 
 11  home_team_abbr          14108 non-null  object 
 12  visitor_team_abbr       14108 non-null  object 
 13  down                    14108 non-null  int64  
 14  yards_to_go             14108 non-null

In [10]:
import plotly.express as px

# Top intent plays
viz = iq_play.sort_values("IQ_team_play", ascending=False).head(15)
fig = px.bar(viz, x="play_id", y="IQ_team_play", color="game_id",
             title="Top plays by team intent (convergence IQ)")
fig.update_layout(xaxis_title="play_id", yaxis_title="IQ (z-score)")
fig.show()

# Player intent vs frames (volume)
viz2 = iq_play_player.query("frames>=20").nlargest(300, "IQ_mean")
fig2 = px.scatter(viz2, x="frames", y="IQ_mean", hover_data=["player_name","player_position","game_id","play_id"],
                  trendline="ols", title="Player intent per play vs. frame volume")
fig2.update_layout(xaxis_title="frames in play", yaxis_title="mean IQ (per frame)")
fig2.show()
